# Preprocessing Steps for Head Pose Videos

This notebook is for extracting mesh features from `.webm` videos generated by the head pose recording experiment. The result of running this pipeline is a `.json` file for each subject/session in the head pose experiment.

In [ ]:
import numpy as np
import cv2
import os
import fnmatch
import json
!pip install mediapipe
import mediapipe as mp
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.9 MB 2.9 MB/s 
Mounted at /content/drive


# Configure MediaPipe FaceMesh


In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)

# Process Video Pipeline

Define the function to take in a `path` to a video file and output an array of shape (`frames`, `landmarks`, `3`), where `frames` is the number of frames in the video file, `landmarks` is the number of landmarks extracted by the face mesh model, and `3` corresponds to the `x`, `y`, and `z` coordinates for each landmark.

In [ ]:
def extract_mesh_from_video(path):
  # open a video file for video capturing
  cap = cv2.VideoCapture(path)
  out = []
  
  # to see if video capturing has been initialized
  while(cap.isOpened()):
    # return (1) if any frames grabbed (2) grabbed image (empty if ret is false)
    ret, frame = cap.read()
    # Q: why frame could be none?
    if frame is not None:
      # process an RGB image and returns the face landmarks on each detected face
      results = face_mesh.process(frame)
      # check if any faces detected
      if not results.multi_face_landmarks:
        continue
      landmarks = results.multi_face_landmarks[0].landmark
      # store landmarks as an array of arrays
      lm = [[a.x, a.y, a.z] for a in landmarks]
      # 3D tensor that stores landmarks frame by frame
      out.append(lm)
    else:
      break

  if len(out) > 0:
    out = np.reshape(np.array(out), (len(out), -1, 3)).tolist()
  return out

Run through all of the video files in the folder and process their data. If `.json` output already exists for the subject ID, then skip processing. (To overwrite existing data, delete the corresponding `.json` file before running this step.)

In [ ]:
# get the path of the webm file
path = '/content/drive/Shareddrives/URSI 2022/Eye Tracking ML/webm_pose/'

# store all the file directories
all_files = os.listdir(path)
print(len(all_files))

# get unique subjects
unique_subjects = set([filepath.split('_')[0] for filepath in os.listdir(path)])
print(unique_subjects)

for subject in unique_subjects:
  all_data = {}

  print(subject)

  # check if the subject json file already exists. if so, skip the remainning body
  if os.path.isfile('/content/drive/Shareddrives/URSI 2022/Eye Tracking ML/json_pose/'+subject+'.json'):
    continue
   
  subject_data = []
  # go through all file directories in the webm file, find those that start with the subject name
  subject_files = fnmatch.filter(all_files, subject+'*')
  # manage every single file directory that starts with the subject name
  for filename in subject_files:
    # transform file name into an array
    fileinfo = filename.replace('.','_').split('_')
    # store relevant values
    subject = fileinfo[0]
    meshfeatures = extract_mesh_from_video(path + filename)
    # create and append a dictionary to the exisiting array
    subject_data.append({
        'features': meshfeatures 
    })
  # once the last for loop is over, assign the subject_data array as the value to the subject key
  all_data[subject] = subject_data

  # export the json file for the subject to the drive
  with open('/content/drive/Shareddrives/URSI 2022/Eye Tracking ML/json_pose/'+subject+'.json', 'w') as file:
    json.dump(all_data, file)

56
{'cmxk63s9', 'pvryvfwa', 'cwt70a0t', 'k1ofdrbc', '1ze2lppo', 'dglf69wf', '9d9vbql0', 'w9v7s1qs', 'tge4y8ud', '2suq3xfy', 'yj7dyk2e', 'lhrhyyon', 'vrx1gw70', '26n5e9ov', '2qxwc8jg', 'trpy4s76', 'u1k9g5s0', 'myj2adhw', 'q07qdfbn', 'kxeoqbnd', 'c56h6n8x', 'bor9krjs', 'y1su0hzk', 'rdgs44u1', '0phtxs0b', '6hdlxqjp', '095e0k9q', 'stp5srte', 'lxbcmkkp', 'f35x3ycv', 'q9f3nczs', '76u9jwyr', 'ef1w751t', 'lkww796q', '03r8jhv6', 'nrw7klk6', 'f1z4jwv2', 'k1w0pgny', '1yt83duh', 'bcafboa7', 'nok0norz', '37gnx93o', '2yrcw5xy', '464qojj0', 'bxjgo5x9', 'shvb0ync', 'dvmwxexg', 'nwwz56wd', '0rl41qzs', '1776mmn2', 'zzxpetp1', 's7e192r1', 'tvrkm302', 'gjq0x5c8', '7rym69s3', 'unbhythd'}
cmxk63s9
pvryvfwa
cwt70a0t
k1ofdrbc
1ze2lppo
dglf69wf
9d9vbql0
w9v7s1qs
tge4y8ud
2suq3xfy
yj7dyk2e
lhrhyyon
vrx1gw70
26n5e9ov
2qxwc8jg
trpy4s76
u1k9g5s0
myj2adhw
q07qdfbn
kxeoqbnd
c56h6n8x
bor9krjs
y1su0hzk
rdgs44u1
0phtxs0b
6hdlxqjp
095e0k9q
stp5srte
lxbcmkkp
f35x3ycv
q9f3nczs
76u9jwyr
ef1w751t
lkww796q
03r8jhv6
nrw7klk6


In [ ]:
unique_subjects

{'03r8jhv6',
 '095e0k9q',
 '0phtxs0b',
 '0rl41qzs',
 '1776mmn2',
 '1yt83duh',
 '1ze2lppo',
 '26n5e9ov',
 '2qxwc8jg',
 '2suq3xfy',
 '2yrcw5xy',
 '37gnx93o',
 '464qojj0',
 '6hdlxqjp',
 '76u9jwyr',
 '7rym69s3',
 '9d9vbql0',
 'bcafboa7',
 'bor9krjs',
 'bxjgo5x9',
 'c56h6n8x',
 'cmxk63s9',
 'cwt70a0t',
 'dglf69wf',
 'dvmwxexg',
 'ef1w751t',
 'f1z4jwv2',
 'f35x3ycv',
 'gjq0x5c8',
 'k1ofdrbc',
 'k1w0pgny',
 'kxeoqbnd',
 'lhrhyyon',
 'lkww796q',
 'lxbcmkkp',
 'myj2adhw',
 'nok0norz',
 'nrw7klk6',
 'nwwz56wd',
 'pvryvfwa',
 'q07qdfbn',
 'q9f3nczs',
 'rdgs44u1',
 's7e192r1',
 'shvb0ync',
 'stp5srte',
 'tge4y8ud',
 'trpy4s76',
 'tvrkm302',
 'u1k9g5s0',
 'unbhythd',
 'vrx1gw70',
 'w9v7s1qs',
 'y1su0hzk',
 'yj7dyk2e',
 'zzxpetp1'}